In [ ]:
from test_utils import *
from swayam import Agent

### Function Calling with Swayam Tools

Let's now start to link the functionality of an LLM Agent with a locally available function call.

#### Let's see what happens without a function call

In [ ]:
from swayam import Swayam
Swayam.execute("What's the weather like in Boston?")

### Some Dummy Functions

Let's create some dummy functions

In [ ]:
from enum import Enum
class UnitEnum(Enum):
    CELSIUS = "celsius"
    FAHRENHEIT = "fahrenheit"

import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit):
    import random
    i = random.randint(-10, 40)
    if unit.lower() == "celsius":
        return f"{i}°C"
    elif unit.lower() == "fahrenheit":
        return f"{i*9/5 + 32}°F"
    else:
        raise ValueError("Invalid unit")
    
def calculate(expression):
    return eval(expression)

def search_email(text):
    import re
    emails = re.findall(r'([\w\.-]+@[\w\.-]+)', text)
    if emails:
        return emails
    else:
        return None

## Tool Creation

As a part of an LLM workflow, we want to trigger the above local function based on response from the LLM (using function calling).

Before we can do that, we need to create a sort of bridge between a function from the view point of an LLM vs what it means in Python.

Here we see how we can create convert a function into a Tool.

### ToolBuilder

We use the tool builder to wrap the function created above.
1. Provide the function callable and describe it.
2. Add all arguments, while describing their types and purposes.

In [ ]:
from swayam import Tool
builder = Tool.tool_builder(get_current_weather, desc="Get the current weather in a given location")
builder.add_field("location", type=str, desc="Location to get weather information for")
builder.add_field("unit", type=UnitEnum, desc="Unit of temperature", default=UnitEnum.CELSIUS)
get_current_weather_tool = builder.build()

builder = Tool.tool_builder(calculate, desc="Evaluates a mathematical expression")
builder.add_field("expression", type=str, desc="The expression to be evaluated.")
calc_tool = builder.build()

builder = Tool.tool_builder(search_email, desc="Searches email addresses in a given text")
builder.add_field("text", type=str, desc="The target text in which emails are to be searched.")
search_email_tool = builder.build()


### Tool Execution

The key thing to keep in mind is that all arguments are passed as keyword arguments irrespective of the underlying function.

In [ ]:
get_current_weather_tool(location="dummy")

In [ ]:
get_current_weather_tool(**{"location": "dummy"})

In [ ]:
get_current_weather_tool(**{"location": "dummy", "unit":"fahrenheit"})

In [ ]:
calc_tool(expression="2 + 5")

In [ ]:
search_email_tool(text="This is a sample text with a@b.com address")

In [ ]:
search_email_tool(text="This is a sample text with no email address")

### Tool Definition

One of the key benefits of creating a Tool is that it self describes itself, in a way compatible with an LLM.

In [ ]:
print(get_current_weather_tool.definition)

In [ ]:
print(calc_tool.definition)

In [ ]:
print(search_email_tool.definition)

Let's create the list of tool definitions to be shared with the LLM.

In [ ]:
tools = [get_current_weather_tool, calc_tool, search_email_tool]

Let's create a tools dictionary so that once we know the name of the tool from LLM, we can call it with arguments returned from the LLM.

In [ ]:
from swayam import Agent, Prompt

prompt = Prompt.as_system("You are a useful assistant. You are provided with a list of tools to help users with their queries.", tools=tools)
agent = Agent(system_prompt=prompt)

In [ ]:
from swayam import Prompt
agent.execute("What's the weather like in Boston?", reset_report=True)

In [ ]:
tools_dict = {tool.name: tool for tool in tools}

In [ ]:
from swayam import Prompt
calls = agent.execute("What's the weather like in Boston?").tool_calls
for call in calls:
    print(tools_dict[call.function.name](**json.loads(call.function.arguments)))

In [ ]:
calls = agent.execute("What is 7 *30 + 13?").tool_calls
for call in calls:
    print(tools_dict[call.function.name](**json.loads(call.function.arguments)))

In [ ]:
text = "This is a sample text with a@b.com and c@d.com addresses"
calls = agent.execute(f"Find whether there is an email address in the following text marked with triple backticks \n ```{text}```").tool_calls
for call in calls:
    print(tools_dict[call.function.name](**json.loads(call.function.arguments)))